In [0]:
from aeso_api import aeso
import pandas as pd
from datetime import datetime, timedelta
import pytz
from pyspark.sql.functions import col, expr, current_timestamp
import dlt

aeso_api_key = 'be5f320c48914d229551e20990daed49'
aeso_instance = aeso(aeso_api_key)


@dlt.view
def aeso_pool_price_bronze(name = "aeso_pool_price_bronze"):
    # Get Current Date, and Current Date + 1
    utc_time = pytz.utc.localize(datetime.utcnow())
    calgary_time = utc_time.astimezone(pytz.timezone("America/Edmonton"))

    current_date = calgary_time.strftime("%Y-%m-%d")
    tomorrow_date = (calgary_time + timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"Current Date: {current_date}")
    print(f"Tomorrow Date: {tomorrow_date}")
    resp = aeso_instance.get_pool_price_report(
        start_date=current_date, end_date=tomorrow_date
    )
    pdf = pd.DataFrame(resp)
    df = spark.createDataFrame(pdf)
    df = df

    return df.withColumn("ingest_timestamp", current_timestamp())

